In [1]:
import random
import math
from math import isclose
from collections import deque
from functools import partial
import pandas as pd
import matplotlib.pyplot as plt 
import heapq

In [2]:
'''
Retorna uma VA exponencial com taxa lambda, passada como argumento
'''
def generate_exp(lambda_param):

  # Gera a variavel aleatoria uniforme no intervalo [0.0, 1.0)
  u0 = random.random()

  # Pega amostra da exponencial
  x0 = - math.log(u0) / lambda_param
  
  # Retorna a amostra
  return x0

In [3]:
def exp02():
  return generate_exp(0.2)


def exp04():
  return generate_exp(0.4)


def exp06():
  return generate_exp(0.6)


def exp08():
  return generate_exp(0.8)


def exp09():
  return generate_exp(0.9)


def exp10():
  return generate_exp(1.0)


def exp001():
  return generate_exp(0.01)

In [4]:
"""
Distribuição com apenas um valor: 2
"""
def every2sec():
  return 2.0


"""
Classe para gerar uma distribuição com 2 numeros alternados entre si
Ex: 1, 2, 1, 2, 1 ...
"""
class AlternateDist:
  def __init__(self,value1, value2):
    self.value1 = value1
    self.value2 = value2
    self.alternate = value1
  # Gera um valor e já alterna para a proxima amostra
  def sample(self):
    value = self.alternate
    if self.alternate == self.value1:
      self.alternate = self.value2
      return value
    else:
      self.alternate = self.value1
      return value

In [5]:
'''
- Recebe: 
  - número de rodadas, tamanho da amostra, taxa lambda, seed
- Retorna:
  - uma lista de rodadas com números aleatorios gerados pela amostra exponencial
'''
def get_samples_rounds(number_rounds, number_samples, lambda_generic, seed):
  random.seed(seed)
  rounds=[]
  for x in range(number_rounds):
    samples=[]
    for y in range(number_samples):
      samples.append(generate_exp(lambda_generic))
    rounds.append(samples)
  return rounds

#Exemplo 1

'''
- Recebe:
  - lista A de rodadas gerada pela função get_samples_rounds
  - lista B de rodadas gerada pela função get_samples_rounds
- Retorna:
  - "There are equal values", caso tenha encontrado um valor em comum nas lista
  - "No equal values", caso as listas não possuem valores em comum
'''
def simple_test(roundsA, roundsB):
  size_rounds_list = len(roundsA)
  for x in range(size_rounds_list-1):
    list_turn = roundsA[x]
    for y in range(size_rounds_list):
      list_verify = roundsB[y]
      result = list(set(list_turn).intersection(list_verify))
      if len(result) > 0:
        return "There are equal values"
  return "No equal values" 

# Exemplo 2
'''
- Recebe:
  - número de amostras de cada lista. A função gera apenas duas listas para 
    testar a correlação
- Retorna:
  - o valor da correlação de Pearson
'''
def test_pearson(n_samples, seed1, seed2):
  
  rounds = []  
  samples1 = []
  samples2 = []

  random.seed(seed1)
  for x in range(n_samples):
    samples1.append(random.random())
  rounds.append(samples1)
  random.seed(seed2)
  for x in range(n_samples):
    samples2.append(random.random())
  rounds.append(samples2)

  x = pd.Series(rounds[0])
  y = pd.Series(rounds[1])
  result = x.corr(y)
  return result

In [6]:
'''
The Pearson correlation coefficient measures the linear association between variables. 
Its value can be interpreted like so:

+1 - Complete positive correlation 

+0.8 - Strong positive correlation

+0.6 - Moderate positive correlation

0 - no correlation whatsoever

-0.6 - Moderate negative correlation

-0.8 - Strong negative correlation

-1 - Complete negative correlation "  

fonte: https://stackabuse.com/calculating-pearson-correlation-coefficient-in-python-with-numpy/

'''

# Media dos indices 
values_corr = test_pearson(10000, 0, 1)
print(round((values_corr),6))

0.009506


In [7]:
ARRIVAL = 1
END_OF_SERVICE = 0
MU = 1.0
IDLE = 0
BUSY = 1
FIFO = 0
LIFO = 1

In [8]:
"""
Representam os eventos de chegada ou fim de serviço
"""
class Event:
  # Construtor da classe Evento
  def __init__(self, event_type, t, client_id):
    # Chegada ou fim de serviço
    self.event_type = event_type
    # Instante que o evento ocorre
    self.t = t
    # ID do cliente ao qual o evento esta relacionado
    self.client_id = client_id
    # Define como a classe vai ser impressa
  def __repr__(self) -> str:
    if self.event_type == ARRIVAL:
      type_str = "ARRIVAL"
    else:
      type_str = "END_OF_SERVICE"
    return f"event_type: {type_str}\nt: {self.t}\nclient_id: {self.client_id}"

"""
O cliente que chega e é servido.
Serve para guardar o tempo de espera até o serviço.
"""
class Client:
  def __init__(self, arrival_event, color):
    self.id = arrival_event.client_id
    self.arrival_time = arrival_event.t
    # tempo de espera até ser servido, deve ser alterado quando for atendido
    self.color = color
    self.waiting_time = 0.0
  # Define como a classe vai ser impressa
  def __repr__(self) -> str:
    return f"id: {self.id}\narrival_time: {self.arrival_time}\nwaiting_time: {self.waiting_time}"


def create_client(event, color):
  # Cria uma instancia da classe Cliente
  new_client = Client(event, color)
  # Retorna referencia para o cliente criado
  return new_client

# TESTES
event = Event(ARRIVAL, 0.1, 0)
print(f"{event}\n")
color = "%06x" % random.randint(0, 0xFFFFFF)
client = create_client(event, color)
print(client)

event_type: ARRIVAL
t: 0.1
client_id: 0

id: 0
arrival_time: 0.1
waiting_time: 0.0


In [9]:
'''
- Recebe o t(tempo atual), arrival_dist (função que gera uma amostragem da 
distribuição desejada para a chegada) e o client_id (id do cliente que chegou)
- Retorna o evento gerado de chegada
'''
def generate_arrival(t, arrival_dist, client_id):
  arrival = Event(ARRIVAL, t + arrival_dist(), client_id)
  return arrival


'''
- Recebe o t(tempo atual), arrival_dist (função que gera uma amostragem da 
distribuição desejada para o tempo de serviço) e o client_id (id do cliente que
vai ser servido)
- Retorna o evento gerado de fim de serviço
'''
def generate_end_of_service(t, service_dist, client_id):
  end_of_service = Event(END_OF_SERVICE, t + service_dist(), client_id)
  return end_of_service

# TESTES
'''
Função auxiliar para imprimir os atributos de um evento
'''
def print_event(event, event_name):
  print(f"{event_name}.client_id: {event.client_id}")
  print(f"{event_name}.t: {event.t}")
  print(f"{event_name}.event_type: {event.event_type}")

# Chegada
arrival = generate_arrival(1, partial(generate_exp, 0.3), 0)
print_event(arrival, "arrival")
print("")

# Fim de serviço
end_of_service = generate_end_of_service(10, partial(generate_exp, 0.6), 1)
print_event(end_of_service, "end_of_service")

arrival.client_id: 0
arrival.t: 2.1912470594540676
arrival.event_type: 1

end_of_service.client_id: 1
end_of_service.t: 10.22901326722026
end_of_service.event_type: 0


In [10]:
class EventsList:
  def __init__(self):
    self.events = []
  def add(self, event):
    # Temos que adicionar o tempo e o id para critério de comparação para a heap
    heapq.heappush(self.events, (event.t, event.event_type, event.client_id, event))
  def pop(self):
    # Pega o evento que ocorre mais cedo, já que a lista esta ordenada pelos tempos
    t, event_type, id, event = heapq.heappop(self.events)
    return event


def get_event_time(event):
  return event.t

# TESTES - Testando a ordem dentro da lista

# Gerando eventos
exp03 = partial(generate_exp, 0.3)
event0 = generate_arrival(20.0, exp03, 0)
event1 = generate_arrival(40.0, exp03, 1)
event2 = generate_arrival(60.0, exp03, 2)
event3 = generate_arrival(62.0, exp03, 3)
event4 = generate_end_of_service(0.0, exp03, 4)
event5 = generate_end_of_service(100.0, exp03, 5)

# Adicionando eventos na lista
events_list = EventsList()
events_list.add(event0)
events_list.add(event1)
events_list.add(event2)
events_list.add(event3)
events_list.add(event4)
events_list.add(event5)
# Verificando ordem dos elementos na lista
print("Elementos na lista:\n")
for t, event_type, id, event in events_list.events:
  print_event(event, f"event{event.client_id}")
  print("")
print("----------------------")

while(events_list.events):
  popped_event = events_list.pop()
  print(f"Removeu: Evento {popped_event.client_id} - Tempo: {popped_event.t}")

Elementos na lista:

event4.client_id: 4
event4.t: 6.8238647618724775
event4.event_type: 0

event0.client_id: 0
event0.t: 27.514042568626518
event0.event_type: 1

event2.client_id: 2
event2.t: 67.65642987636367
event2.event_type: 1

event3.client_id: 3
event3.t: 62.68180503424469
event3.event_type: 1

event1.client_id: 1
event1.t: 40.5619047668402
event1.event_type: 1

event5.client_id: 5
event5.t: 103.99689851486175
event5.event_type: 0

----------------------
Removeu: Evento 4 - Tempo: 6.8238647618724775
Removeu: Evento 0 - Tempo: 27.514042568626518
Removeu: Evento 1 - Tempo: 40.5619047668402
Removeu: Evento 3 - Tempo: 62.68180503424469
Removeu: Evento 2 - Tempo: 67.65642987636367
Removeu: Evento 5 - Tempo: 103.99689851486175


In [11]:
class ClientsQueue:
  def __init__(self):
    self.clients = deque([])
  def add(self, client):
    self.clients.append(client)
  def pop(self, queue_discipline):
    if queue_discipline == FIFO:
      return self.clients.popleft()  # Se for FIFO tira do inicio da lista
    else:
      return self.clients.pop()  # Se for LIFO tira do fim da lista


# TESTES

# Gerando chegadas
arrival1 = Event(ARRIVAL, 0.2, 0)
arrival2 = Event(ARRIVAL, 1.7, 1)
arrival3 = Event(ARRIVAL, 0.1, 2)
arrivals = [arrival1, arrival2, arrival3]

# Criando a fila de clientes
clients_queue = ClientsQueue()
# Gerando clientes a partir das chegadas e adicionando na fila
color = "%06x" % random.randint(0, 0xFFFFFF)
for arrival in arrivals:
  client = create_client(arrival, color)
  clients_queue.add(client)

# Imprimindo os clientes na fila
print("FILA")
for client in clients_queue.clients:
  print(f"{client}\n")
print("---------------------")

# Backup temporario de clientes
temp_list = clients_queue.clients.copy()
# Saida FIFO
print("Saida FIFO")
while(clients_queue.clients):
  popped = clients_queue.pop(FIFO)
  print(f"{popped}\n")
print("---------------------")

# Retornando os clientes para a fila
for client in temp_list:
  clients_queue.add(client)
# Saida LIFO
print("Saida LIFO")
while(clients_queue.clients):
  popped = clients_queue.pop(LIFO)
  print(f"{popped}\n")

FILA
id: 0
arrival_time: 0.2
waiting_time: 0.0

id: 1
arrival_time: 1.7
waiting_time: 0.0

id: 2
arrival_time: 0.1
waiting_time: 0.0

---------------------
Saida FIFO
id: 0
arrival_time: 0.2
waiting_time: 0.0

id: 1
arrival_time: 1.7
waiting_time: 0.0

id: 2
arrival_time: 0.1
waiting_time: 0.0

---------------------
Saida LIFO
id: 2
arrival_time: 0.1
waiting_time: 0.0

id: 1
arrival_time: 1.7
waiting_time: 0.0

id: 0
arrival_time: 0.2
waiting_time: 0.0



In [12]:
"""
client -> Cliente a ser servido
t -> Instante atual do sistema
service_dist -> distribuição do tempo de serviço
events_list -> lista de eventos para adicionar o fim do serviço
"""
def serve_client(client, t, service_dist, events_list):
  # Registra o tempo de espera do serviço
  client.waiting_time = t - client.arrival_time
  # Cria evento do fim do serviço
  end_of_service_event = generate_end_of_service(t, service_dist, client.id)
  # Adiciona evento do fim do servico a lista de eventos
  events_list.add(end_of_service_event)


# TESTES
# Cria a lista de eventos
events_list = EventsList()
# Gera os eventos
exp03 = partial(generate_exp, 0.3)
event0 = generate_arrival(90.0, exp03, 0)
event1 = generate_arrival(40.0, exp03, 1)
event2 = generate_arrival(60.0, exp03, 2)
event3 = generate_arrival(62.0, exp03, 3)
events = [event0,event1,event2,event3]
# Popula a lista de eventos
for event in events:
  events_list.add(event)
print("Lista de eventos ANTES do serviço:")
print(f"{events_list.events}\n")
# Pega a primeira chegada e cria um cliente
first_event = events_list.pop()
color = "%06x" % random.randint(0, 0xFFFFFF)

client = create_client(first_event, color)
print("Cliente ANTES do serviço:")
print(f"{client}\n")
# Serve o cliente
serve_client(client, 65.0, exp03, events_list)
print("Lista de eventos DEPOIS do serviço:")
print(f"{events_list.events}\n")
print("Cliente DEPOIS do serviço:")
print(client)

Lista de eventos ANTES do serviço:
[(46.8609922712309, 1, 1, event_type: ARRIVAL
t: 46.8609922712309
client_id: 1), (69.50320490523724, 1, 3, event_type: ARRIVAL
t: 69.50320490523724
client_id: 3), (60.230348727581905, 1, 2, event_type: ARRIVAL
t: 60.230348727581905
client_id: 2), (96.43665556179093, 1, 0, event_type: ARRIVAL
t: 96.43665556179093
client_id: 0)]

Cliente ANTES do serviço:
id: 1
arrival_time: 46.8609922712309
waiting_time: 0.0

Lista de eventos DEPOIS do serviço:
[(60.230348727581905, 1, 2, event_type: ARRIVAL
t: 60.230348727581905
client_id: 2), (67.46217817254852, 0, 1, event_type: END_OF_SERVICE
t: 67.46217817254852
client_id: 1), (96.43665556179093, 1, 0, event_type: ARRIVAL
t: 96.43665556179093
client_id: 0), (69.50320490523724, 1, 3, event_type: ARRIVAL
t: 69.50320490523724
client_id: 3)]

Cliente DEPOIS do serviço:
id: 1
arrival_time: 46.8609922712309
waiting_time: 18.1390077287691


In [13]:
T_PERCENTILE = 1.960
NSAMPLES = 10000
# Intervalo de confiança da t-student
def tstudent_ci(mean, variance):
  # Verifica se a media e variancia tem valores nulos (ou bem proximos disso)
  if isclose(mean, 0, abs_tol=1e-9):
    mean = 0.0
  if isclose(variance, 0, abs_tol=1e-9):
    variance = 0.0
  # Se variancia e media forem nulas
  if not(mean or variance):
    return 0.0, 0.0, 0.0
  # desvio padrao (raiz quadrada da variancia)
  std_dev = math.sqrt(variance)
  # metade do intervalo
  term = T_PERCENTILE * (std_dev/math.sqrt(NSAMPLES))
  # limite superior do intervalo
  upper_limit = mean + term
  # limite inferior do intervalo
  lower_limit = mean - term
  # precisao do intervalo de confiança (metade do intervalo / centro do intervalo)
  precision = term / mean 
  return upper_limit, lower_limit, precision

In [14]:
CHI_0975_PERCENTILE = 3357.658
CHI_0025_PERCENTILE = 3044.13

'''
- Recebe a variância estimada, o número de coletas por rodada da simulação e o
número de rodadas da simulação.
- Retorna o intervalo de confiança da variância
'''
def chi_square(variance, k, n):
  # Se a variancia é zero, nao há intervalo, é apenas zero
  if variance == 0.0:
    upper_limit, lower_limit, precision = 0.0, 0.0, 0.0
  else:
    upper_limit = (n - 1) * variance / CHI_0025_PERCENTILE
    lower_limit = (n - 1) * variance / CHI_0975_PERCENTILE
    # Aproximadamente 0.05, já que esse foi o motivo para escolher 3200 rodadas
    precision = (CHI_0975_PERCENTILE - CHI_0025_PERCENTILE) / (CHI_0975_PERCENTILE + CHI_0025_PERCENTILE )
  return upper_limit, lower_limit, precision

In [15]:
'''
Classe para calcular os estimadores de média e variancia de 
do tempo de espera na fila e numero medio de pessoas na fila de espera
'''
class Estimator:
  def __init__(self):
    # Soma das amostras
    self.samples_sum = 0.0
    # Soma dos quadrados das amostras
    self.squares_sum = 0.0
    # Numero de amostras
    self.n = 0
  def add_sample(self, sample):
    self.samples_sum += sample
    self.squares_sum += (sample**2)
    self.n += 1
  def mean(self):
    return self.samples_sum / self.n
  def variance(self):
    term1 = self.squares_sum / (self.n - 1)
    term2 = (self.samples_sum**2) / (self.n * (self.n - 1))
    return term1 - term2
  def tstudent_ci(self):
    return tstudent_ci(self.mean(), self.variance())
  def chi_square_var(self, round_size, rounds):
    return chi_square(self.variance(), round_size, rounds)
  def chi_square_ci(self, round_size, rounds):
    return chi_square(self.mean(), round_size, rounds)

# TESTE
samples = [0.5, 1, 2, 3, 7.1, 9.6, 10.1, 13.4]
stats = Estimator()
for sample in samples:
  stats.add_sample(sample)
print(stats.mean())
print(stats.variance())

5.8374999999999995
23.682678571428582


In [16]:
"""
Classe para calcular o número medio de pessoas na fila de espera
durante uma rodada de simulação usando a área do gráfico clientes x tempo.
"""
class NqueueAreaEstimator:
  def __init__(self):
    # Soma das areas
    self.nqueue_area_sum = 0.0
    # Soma dos intervalos de tempo
    self.dt_sum = 0.0
  def add_sample(self, nqueue, dt):
    self.nqueue_area_sum += nqueue * dt
    self.dt_sum += dt
  def mean(self):
    return self.nqueue_area_sum / self.dt_sum

In [17]:
"""
Classe para calcular a variancia do numero de pessoas na fila de espera
durante uma rodada de simulação usando a pmf.
"""
class NqueuePmfEstimator:
  def __init__(self):
    # Soma dos quadrados dos Nqis coletados vezes o intervalo de tempo
    self.nqueue_squares_sum = 0.0
    # Soma dos Nqis coletados vezes o intervalo de tempo
    self.nqueue_sum = 0.0
    # Soma dos intervalos de tempo
    self.dt_sum = 0.0
  def add_sample(self, nqueue, dt):
    self.nqueue_squares_sum += (nqueue**2) * dt
    self.nqueue_sum += nqueue * dt
    self.dt_sum += dt
  def variance(self):
    second_moment = self.nqueue_squares_sum / self.dt_sum
    first_moment = self.nqueue_sum / self.dt_sum
    return second_moment - (first_moment**2)

In [18]:
'''
- Recebe o número de rodadas, tamanho da rodada, disciplina da fila de 
fregueses, distribuição do tempo entre chegadas e distribuição do tempo de serviço
- Retorna vazio
- arrival_dist gera o tempo para uma chegada
- service_dist gera o tempo para o fim do serviço
'''
def simulate(rounds, round_size, queue_discipline, arrival_dist, service_dist,
             print_values=False, print_graph=False, graph_values=False):
  # tempo inicial
  t = 0.0
  # numero de pessoas na fila de espera
  nqueue = 0
  # Cliente no servidor
  client_in_server = None
  # ID inicial dos clientes
  client_id = 0
  # Cria o evento inicial
  first_arrival = generate_arrival(t, arrival_dist, client_id)
  # Incrementa o identificador de clientes
  client_id += 1
  # Cria a lista de eventos, com o evento inicial
  events_list = EventsList()
  events_list.add(first_arrival)
  # Cria a fila de espera de clientes, vazia
  clients_queue = ClientsQueue()
  # Estimadores de tempo de espera e tamanho da fila
  time_est = Estimator()
  nqueue_est = Estimator()
  # Estimadores da variancia pela t-student
  time_var_est = Estimator()
  nqueue_var_est = Estimator()
  # lista com os tempos de espera nas filas de cada rodada
  roundW = []
  # lista com os valores do estimador de E[W] ao longo do tempo
  W_mean = []
  W_var = []
  # lista com numeros de individuos na fila em cada rodada
  Nq_mean = []
  Nq_var = []
  # Loop das Rodadas
  for i in range(rounds):
    # Numero de clientes servidos na rodada
    served_clients = 0
    # Definindo a cor da rodada
    color = "%06x" % random.randint(0, 0xFFFFFF)
    # Inicia os estimadores de tempo de espera e tamanho da fila da rodada
    round_time_est = Estimator()
    round_nqueue_est = NqueueAreaEstimator()
    round_nqueue_pmf_est = NqueuePmfEstimator()
    while(served_clients < round_size):
      # Seleciona o próximo evento e tira da lista
      event = events_list.pop()
      # Salva o valor de Nq durante o intervalo de tempo ate o evento
      round_nqueue_est.add_sample(nqueue, event.t - t)
      round_nqueue_pmf_est.add_sample(nqueue, event.t - t)
      # Avança o tempo para o instante do evento
      t = event.t
      # Se o evento é uma CHEGADA
      if event.event_type == ARRIVAL:
        # Cria um cliente
        new_client = create_client(event, color)
        # Se o servidor esta LIVRE
        if not client_in_server:
          # Coloca o cliente no servidor
          client_in_server = new_client
          # Registra tempo de espera e adiciona evento do fim do serviço na lista
          serve_client(new_client, t, service_dist, events_list)
        # Se o servidor esta OCUPADO
        else:
          # Adiciona o cliente na fila de espera
          clients_queue.add(new_client)
          # Incrementa numero de clientes na fila de espera
          nqueue += 1
        # Incrementa id de cliente
        client_id += 1
        # Cria nova chegada 
        new_arrival = generate_arrival(t, arrival_dist, client_id)
        # Adiciona nova chegada na lista de eventos
        events_list.add(new_arrival)
      # Se o evento é o FIM DE SERVIÇO
      else:
        # Se o cliente tiver a cor da rodada atual
        if color == client_in_server.color:
          # Salva o tempo de espera do cliente que acabou o serviço
          round_time_est.add_sample(client_in_server.waiting_time)
          # Incrementa numero de clientes que ja foram servidos na rodada
          served_clients += 1
        # Se nao ha NINGUEM na fila de espera
        if nqueue == 0:
          # Muda status do servidor para vazio
          client_in_server = None
        # Se ha ALGUEM na fila de espera
        else:
          # Pega o proximo cliente
          client = clients_queue.pop(queue_discipline)
          # Coloca o cliente no servidor
          client_in_server = client
          # Registra tempo de espera e adiciona evento do fim do serviço na lista
          serve_client(client, t, service_dist, events_list)
          # Decrementa numero de clientes na fila de espera
          nqueue -= 1
    # Calcula a estatisticas de W e Nq da rodada (amostras da rodada)
    round_time_mean = round_time_est.mean()
    round_time_var = round_time_est.variance()
    round_nqueue_mean = round_nqueue_est.mean()
    round_nqueue_var = round_nqueue_pmf_est.variance()
    # Salva as amostras da rodada nos estimadores gerais
    time_est.add_sample(round_time_mean)
    time_var_est.add_sample(round_time_var)
    nqueue_est.add_sample(round_nqueue_mean)
    nqueue_var_est.add_sample(round_nqueue_var)
    # Salva amostras para o grafico, se desejado
    if graph_values:
      W_mean.append(time_est.mean())
      Nq_mean.append(nqueue_est.mean())
      if i > 0:
        W_var.append(time_est.variance())
        Nq_var.append(nqueue_est.variance())
        
  # Obtem os intervalos de confiança e seus centros
  # t-student para media do tempo
  w_mean_upper, w_mean_lower, w_mean_precision = time_est.tstudent_ci()
  w_mean_center = time_est.mean()
  # chi-square para variancia do tempo
  w_var_upper, w_var_lower, w_var_precision = time_est.chi_square_var(round_size, rounds)
  w_var_center = time_est.variance()
  # t-student para variancia do tempo
  w_var_upper2, w_var_lower2, w_var_precision2 = time_var_est.tstudent_ci()
  w_var_center2 = time_var_est.mean()
  # t-student para media de Nq
  nq_mean_upper, nq_mean_lower, nq_mean_precision = nqueue_est.tstudent_ci()
  nq_mean_center = nqueue_est.mean()
  # chi-square para variancia de Nq
  nq_var_upper, nq_var_lower, nq_var_precision = nqueue_est.chi_square_var(round_size, rounds)
  nq_var_center = nqueue_est.variance()
  # t-studente para variancia de Nq
  nq_var_upper2, nq_var_lower2, nq_var_precision2 = nqueue_var_est.tstudent_ci()
  nq_var_center2 = nqueue_var_est.mean()

  if print_values:
    print("(a) Média de W")
    print(f"Intervalo de Confiança: {w_mean_lower:.3f} a {w_mean_upper:.3f}")
    print(f"Precisão: {w_mean_precision * 100:.2f}%")
    print("------------------")
    print("(b) Variância de W - Chi-Square")
    print(f"Intervalo de Confiança: {w_var_lower:.3f} a {w_var_upper:.3f}")
    print(f"Precisão: {w_var_precision * 100:.2f}%")
    print("------------------")
    print("(b) Variância de W - t-Student")
    print(f"Intervalo de Confiança: {w_var_lower2:.3f} a {w_var_upper2:.3f}")
    print(f"Precisão: {w_var_precision2 * 100:.2f}%")
    print("------------------")
    print("(c) Média de Nq")
    print(f"Intervalo de Confiança: {nq_mean_lower:.3f} a {nq_mean_upper:.3f}")
    print(f"Precisão: {nq_mean_precision * 100:.2f}%")
    print("------------------")
    print("(d) Variância de Nq - Chi-Square")
    print(f"Intervalo de Confiança: {nq_var_lower:.3f} a {nq_var_upper:.3f}")
    print(f"Precisão: {nq_var_precision * 100:.2f}%")
    print("------------------")
    print("(d) Variância de Nq - t-Student")
    print(f"Intervalo de Confiança: {w_var_lower2:.3f} a {w_var_upper2:.3f}")
    print(f"Precisão: {w_var_precision2 * 100:.2f}%")
  else:
    if queue_discipline == FIFO:
      discipline = "FCFS"
    else:
      discipline = "LCFS"
    return {"time_mean": [discipline, w_mean_lower, w_mean_upper, w_mean_precision, w_mean_center],
            "time_var_chi": [discipline, w_var_lower, w_var_upper, w_var_precision, w_var_center],
            "time_var_t": [discipline, w_var_lower2, w_var_upper2, w_var_precision2, w_var_center2],
            "nqueue_mean": [discipline, nq_mean_lower, nq_mean_upper, nq_mean_precision, nq_mean_center],
            "nqueue_var_chi": [discipline, nq_var_lower, nq_var_upper, nq_var_precision, nq_var_center],
            "nqueue_var_t": [discipline, nq_var_lower2, nq_var_upper2, nq_var_precision2, nq_var_center2],
            "time_graph_values": [W_mean, W_var],
            "nqueue_graph_values": [Nq_mean, Nq_var]}

In [19]:
random.seed(0)
simulate(3300, 1000, FIFO, exp001, exp10, print_values=True)

(a) Média de W
Intervalo de Confiança: 0.010 a 0.010
Precisão: 0.90%
------------------
(b) Variância de W - Chi-Square
Intervalo de Confiança: 0.000 a 0.000
Precisão: 4.90%
------------------
(b) Variância de W - t-Student
Intervalo de Confiança: 0.020 a 0.020
Precisão: 1.55%
------------------
(c) Média de Nq
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.91%
------------------
(d) Variância de Nq - Chi-Square
Intervalo de Confiança: 0.000 a 0.000
Precisão: 4.90%
------------------
(d) Variância de Nq - t-Student
Intervalo de Confiança: 0.020 a 0.020
Precisão: 1.55%


In [20]:
simulate(3300, 1000, FIFO, every2sec, every2sec, print_values=True)

(a) Média de W
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.00%
------------------
(b) Variância de W - Chi-Square
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.00%
------------------
(b) Variância de W - t-Student
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.00%
------------------
(c) Média de Nq
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.00%
------------------
(d) Variância de Nq - Chi-Square
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.00%
------------------
(d) Variância de Nq - t-Student
Intervalo de Confiança: 0.000 a 0.000
Precisão: 0.00%


In [21]:
random.seed(0)

# Distribuicao alternada (1 e 2 segundos)
one_and_two = AlternateDist(1, 2)

# Serviço a cada 1.5 segundo
def every1_5sec():
  return 1.5

simulate(3300, 1000, FIFO, one_and_two.sample, every1_5sec, print_values=True)

(a) Média de W
Intervalo de Confiança: 0.250 a 0.250
Precisão: 0.00%
------------------
(b) Variância de W - Chi-Square
Intervalo de Confiança: 0.000 a 0.000
Precisão: 4.90%
------------------
(b) Variância de W - t-Student
Intervalo de Confiança: 0.063 a 0.063
Precisão: 0.00%
------------------
(c) Média de Nq
Intervalo de Confiança: 0.167 a 0.167
Precisão: 0.00%
------------------
(d) Variância de Nq - Chi-Square
Intervalo de Confiança: 0.000 a 0.000
Precisão: 4.90%
------------------
(d) Variância de Nq - t-Student
Intervalo de Confiança: 0.063 a 0.063
Precisão: 0.00%
